Pontificia Universidad Católica de Chile <br>
Departamento de Ciencia de la Computación <br>
2025 - Bimestre 5 <br>


<h1><center> Procesamiento de Lenguaje Natural

Tarea 3: Transformers </center></h1>
        **Profesor**: Marcelo Mendoza<br>

---

# Integrantes
* Estudiante 1: Jason Sepulveda
* Estudiante 2: n/a

# Instrucciones

* Deberás entregar SOLO el archivo .ipynb.
* Se te dará puntaje tanto por código como por la manera en la que respondas las preguntas planteadas.
* El notebook debe tener todas las celdas de código ejecutadas.

# Librerías

In [1]:
#Incorpore las librerías en esta sección
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# Introducción

El conjunto de datos utilizado en este estudio se construyó a partir de tweets obtenidos a través de la API de Twitter y filtrados utilizando un léxico de discurso de odio proporcionado por Hatebase.org. Inicialmente, se identificaron y recopilaron tweets que contenían términos específicos catalogados como discurso de odio por la comunidad. Este proceso resultó en una muestra de 85.4 millones de tweets, provenientes de 33,458 usuarios de Twitter.

Posteriormente, se realizó una selección aleatoria de tweets de esta muestra inicial para su análisis detallado. Estos tweets fueron evaluados manualmente por trabajadores de CrowdFlower (CF), quienes clasificaron cada tweet en una de tres categorías: discurso de odio, lenguaje ofensivo pero no discurso de odio, y ni ofensivo ni discurso de odio.

**Estructura del dataset**

Para el desarrollo de la tarea, se deben concentrar en dos columnas:

* tweet: Contiene la sentencia que debe ser evaluada.
* label: Clasificación asociada a cada sentencia de la variable tweet.

Respecto a la variable label, se tiene la siguiente codificación:

* 0 - discurso de odio
* 1 - lenguaje ofensivo
* 2 - ninguno

El conjunto de datos ya está divido en train (df_train) y test (df_test)

# 1 - Entrenar clasificador con RoBERTa (10 puntos)

Utilizando el modelo RoBERTa-base, construya un clasificador para la tarea planteada.

Calcule las métricas Precision, Recall y F1-score para el conjunto test (general y por clase). Comente sus resultados.

Considere un entrenamiento por 3 épocas.

**Observación**

* Debe trabajar con GPU. El tiempo de entrenamiento es de 20 minutos, aproximadamente. Se recomienda guardar el modelo para no repetir entrenamiento.
* Respecto a la implementación desarrollada en la **ayudantía 3**, deben modificar la función ```compute_metrics```. A continuación, se presenta la función que debe ser utilizada:

```
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return f1.compute(references=labels, predictions=predictions, average='micro')
```  

Respuesta:

In [2]:
!pip install transformers

In [11]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix

import torch
from torch import cuda
from torch.utils.data import Dataset, DataLoader

import transformers
from transformers import RobertaModel, RobertaTokenizer, BertModel, BertTokenizer, DistilBertModel, DistilBertTokenizer

import json
from tqdm import tqdm

import logging
logging.basicConfig(level=logging.ERROR)

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
    set_seed,
)
from datasets import DatasetDict
from torch.nn import functional as F

import pathlib
from tqdm.auto import tqdm

from datasets import Dataset, Value, ClassLabel, Features
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

In [4]:
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cuda'

In [5]:
#paths
path_train = '/content/w6_df_test.csv'
path_test = '/content/w6_df_test.csv'

df_train = pd.read_csv(path_train)
df_test = pd.read_csv(path_test)

print('='*40)
print(df_train.head())
print('='*40)
print(df_test.head())




   Unnamed: 0  count  hate_speech  offensive_language  neither  label  \
0       16183      3            0                   3        0      1   
1        3239      3            0                   0        3      2   
2       25097      3            0                   3        0      1   
3         365      3            0                   3        0      1   
4       16493      3            0                   3        0      1   

                                               tweet  
0  RT @JimNorton: There is a GIFT SHOP at the 9-1...  
1  @Fabulousjonna_ I need some red green &amp; ye...  
2  this the one&#128076;RT @SumthinNueee hoes don...  
3  "@lildurk_: I can turn yo bitch into a eaterrr...  
4  RT @LUNAraps: smh u got Ebola now RT&#8220;@Br...  
   Unnamed: 0  count  hate_speech  offensive_language  neither  label  \
0       16183      3            0                   3        0      1   
1        3239      3            0                   0        3      2   
2       2509

In [6]:
df_train_filtered = df_train[['tweet', 'label']]
df_test_filtered = df_test[['tweet', 'label']]

In [7]:
# shape
print('Shape train: ', df_train_filtered.shape)
print('Shape test: ', df_test_filtered.shape)


Shape train:  (4957, 2)
Shape test:  (4957, 2)


In [10]:
ds = DatasetDict({
    "train": Dataset.from_pandas(df_train_filtered, preserve_index=False),
    "test":  Dataset.from_pandas(df_test_filtered, preserve_index=False),
})

In [12]:
MODEL_NAME = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_fn(batch):
    return tokenizer(
        batch["tweet"],
        truncation=True,
        max_length=128,
    )

tokenized_ds = ds.map(tokenize_fn, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/4957 [00:00<?, ? examples/s]

Map:   0%|          | 0/4957 [00:00<?, ? examples/s]

In [13]:
from transformers import set_seed
set_seed(0)

In [14]:
id2label = {0: "hate_speech", 1: "offensive_language", 2: "neither"}
label2id = {v: k for k, v in id2label.items()}

In [15]:
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=3,
    id2label=id2label,
    label2id=label2id,
)

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)

    # Global (macro)
    p_macro, r_macro, f1_macro, _ = precision_recall_fscore_support(
        labels, preds, average="macro", zero_division=0
    )

    # Por clase
    p_c, r_c, f1_c, support_c = precision_recall_fscore_support(
        labels, preds, average=None, labels=[0, 1, 2], zero_division=0
    )

    metrics = {
        "precision_macro": p_macro,
        "recall_macro": r_macro,
        "f1_macro": f1_macro,
    }

    for i, lab_id in enumerate([0, 1, 2]):
        name = id2label[lab_id]
        metrics[f"precision_{name}"] = p_c[i]
        metrics[f"recall_{name}"] = r_c[i]
        metrics[f"f1_{name}"] = f1_c[i]
        metrics[f"support_{name}"] = float(support_c[i])

    return metrics

In [29]:
train_args = TrainingArguments(
    report_to="none",
    output_dir='roberta_hatespeech_clf',
    eval_strategy="steps",
    save_strategy = "steps",
    learning_rate=2e-5,
    warmup_steps=250,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.18,
    eval_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model="eval_f1_macro",
)



In [30]:
trainer = Trainer(
    model=model,
    args=train_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/tmp/ipython-input-242998058.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [31]:
trainer.train()

Step,Training Loss,Validation Loss,Precision Macro,Recall Macro,F1 Macro,Precision Hate Speech,Recall Hate Speech,F1 Hate Speech,Support Hate Speech,Precision Offensive Language,Recall Offensive Language,F1 Offensive Language,Support Offensive Language,Precision Neither,Recall Neither,F1 Neither,Support Neither
100,No log,0.228526,0.802553,0.687935,0.700202,0.597222,0.150350,0.240223,286.000000,0.934319,0.971079,0.952344,3838.000000,0.876116,0.942377,0.908039,833.000000
200,No log,0.192780,0.816800,0.760599,0.778918,0.589595,0.356643,0.444444,286.000000,0.949846,0.967170,0.958430,3838.000000,0.910959,0.957983,0.933879,833.000000
300,No log,0.194875,0.790779,0.828801,0.808099,0.482558,0.580420,0.526984,286.000000,0.966880,0.943200,0.954893,3838.000000,0.922900,0.962785,0.942421,833.000000
400,No log,0.146276,0.865451,0.826142,0.843236,0.666667,0.531469,0.591440,286.000000,0.961983,0.975769,0.968827,3838.000000,0.967703,0.971188,0.969443,833.000000


TrainOutput(global_step=465, training_loss=0.22900177330099125, metrics={'train_runtime': 270.7488, 'train_samples_per_second': 54.925, 'train_steps_per_second': 1.717, 'total_flos': 552063853626138.0, 'train_loss': 0.22900177330099125, 'epoch': 3.0})

In [32]:
eval_metrics = trainer.evaluate()
print("\n=== Metrics (Trainer.evaluate) ===")
for k in sorted(eval_metrics.keys()):
    print(f"{k}: {eval_metrics[k]}")

# Reporte clásico (Precision/Recall/F1 por clase + macro/weighted)
pred_out = trainer.predict(tokenized_ds["test"])
y_true = pred_out.label_ids
y_pred = np.argmax(pred_out.predictions, axis=1)

print("\n=== classification_report (TEST) ===")
print(classification_report(
    y_true, y_pred,
    target_names=[id2label[0], id2label[1], id2label[2]],
    digits=4
))


=== Metrics (Trainer.evaluate) ===
epoch: 3.0
eval_f1_hate_speech: 0.6192307692307693
eval_f1_macro: 0.8535633999794953
eval_f1_neither: 0.9705705705705706
eval_f1_offensive_language: 0.9708888601371458
eval_loss: 0.1380302757024765
eval_precision_hate_speech: 0.688034188034188
eval_precision_macro: 0.8744881899276656
eval_precision_neither: 0.9711538461538461
eval_precision_offensive_language: 0.9642765355949627
eval_recall_hate_speech: 0.5629370629370629
eval_recall_macro: 0.8368391847422855
eval_recall_neither: 0.9699879951980792
eval_recall_offensive_language: 0.9775924960917144
eval_runtime: 21.1669
eval_samples_per_second: 234.187
eval_steps_per_second: 7.323
eval_support_hate_speech: 286.0
eval_support_neither: 833.0
eval_support_offensive_language: 3838.0

=== classification_report (TEST) ===
                    precision    recall  f1-score   support

       hate_speech     0.6880    0.5629    0.6192       286
offensive_language     0.9643    0.9776    0.9709      3838
      

In [36]:
trainer.save_model("Modelo Roberta")

### Comentarios
El modelo RoBERTa-base logra un muy buen desempeño general, con una accuracy del 95% y un F1-macro de 0.85. Las clases offensive_language y neither se clasifican casi perfectamente, lo que era esperable dado que cuentan con muchos más ejemplos. En cambio, la clase hate_speech presenta un recall más bajo, lo que indica que varios casos reales no son detectados y se confunden con lenguaje ofensivo. Este comportamiento es típico en datasets desbalanceados y muestra que, aunque el modelo funciona muy bien en general, aún tiene dificultades para identificar correctamente la clase minoritaria.

# 2 - Uso de LLM como clasificador: Zero-shot (8 puntos)

Seleccione una muestra aleatoria de 20 ejemplos por categoría desde el conjunto de test (en total son 60 ejemplos).

Utilice el modelo ```meta-llama/Llama-3.2-3B-Instruct``` para obtener predicciones para cada comentario. Justifique el diseño del prompt.

Ver modelo en: https://huggingface.co/meta-llama/Llama-3.2-3B-Instruct

Calcule las métricas Precision, Recall y F1-score para el conjunto analizado (general y por clase). Comente los resultados.

**Observación**

* Para utilizar este modelo debe registrarse en huggingface y solicitar permiso. Al importarlo, debe ingresar el token de huggingface. Revisar la ayudantía 3.
* Para esta pregunta deben trabajar con GPU.


Respuesta:

In [33]:
from transformers import pipeline

In [34]:
from huggingface_hub import notebook_login

notebook_login()

In [39]:
MODEL_NAME = "meta-llama/Llama-3.2-3B-Instruct"
SEED = 42

In [40]:
df_test = pd.read_csv(path_test)[["tweet", "label"]].dropna()
df_test["label"] = df_test["label"].astype(int)

df_sample = (
    df_test.groupby("label", group_keys=False)
    .apply(lambda g: g.sample(n=20, random_state=SEED))
    .reset_index(drop=True)
)

print(df_sample["label"].value_counts())

label
0    20
1    20
2    20
Name: count, dtype: int64


/tmp/ipython-input-1240571374.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: g.sample(n=20, random_state=SEED))


In [41]:
pipe = pipeline(
    "text-generation",
    model=MODEL_NAME,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

Device set to use cuda:0


In [44]:
import re
LABELS = ["hate_speech", "offensive_language", "neither"]

def build_prompt(text):
    return f"""
You are a text classifier.

Classify the following text into exactly one of these labels:
- hate_speech
- offensive_language
- neither

Text:
"{text}"

Answer ONLY with one label from the list.
Label:
""".strip()

label_regex = re.compile(r"(hate_speech|offensive_language|neither)", re.IGNORECASE)

def predict_label(text):
    prompt = build_prompt(text)

    output = pipe(
        prompt,
        max_new_tokens=10,
        do_sample=False,
    )[0]["generated_text"]

    match = label_regex.search(output.lower())
    if match:
        return match.group(1)
    return "neither"

In [45]:
pred_labels = []

for i, row in df_sample.iterrows():
    pred = predict_label(row["tweet"])
    pred_labels.append(pred)
    if (i + 1) % 10 == 0:
        print(f"{i+1}/60 ejemplos procesados")

df_sample["pred_label_str"] = pred_labels
df_sample["pred_label"] = df_sample["pred_label_str"].map(label2id)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


10/60 ejemplos procesados


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


20/60 ejemplos procesados


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

40/60 ejemplos procesados


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


50/60 ejemplos procesados


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


60/60 ejemplos procesados


In [46]:
y_true = df_sample["label"].values
y_pred = df_sample["pred_label"].values

p_macro, r_macro, f1_macro, _ = precision_recall_fscore_support(
    y_true, y_pred, average="macro", zero_division=0
)

print("\n=== MÉTRICAS (macro) ===")
print("Precision:", p_macro)
print("Recall:", r_macro)
print("F1:", f1_macro)

print("\n=== classification_report ===")
print(classification_report(
    y_true, y_pred,
    target_names=[id2label[0], id2label[1], id2label[2]],
    digits=4
))


=== MÉTRICAS (macro) ===
Precision: 0.1111111111111111
Recall: 0.3333333333333333
F1: 0.16666666666666666

=== classification_report ===
                    precision    recall  f1-score   support

       hate_speech     0.3333    1.0000    0.5000        20
offensive_language     0.0000    0.0000    0.0000        20
           neither     0.0000    0.0000    0.0000        20

          accuracy                         0.3333        60
         macro avg     0.1111    0.3333    0.1667        60
      weighted avg     0.1111    0.3333    0.1667        60



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### Comentarios
El modelo Llama-3.2-3B-Instruct obtiene un desempeño muy bajo (F1-macro = 0.17), ya que predice casi exclusivamente la clase hate_speech y no logra identificar las otras categorías. Esto muestra que, sin fine-tuning, el uso de modelos generativos mediante prompting es poco efectivo para tareas de clasificación, especialmente frente a modelos entrenados específicamente como RoBERTa.

# 3 - Uso de LLM como clasificador: Few-shot (8 puntos)

Utilice el modelo ```meta-llama/Llama-3.2-3B-Instruct``` para obtener predicciones sobre los 60 comentarios seleccionados. Especifique el prompt utilizado, indicando los criterios para seleccionar los ejemplos que serán incluidos en el prompt.

Calcule las métricas Precision, Recall y F1-score para el conjunto analizado (general y por clase). Comente los resultados.


Respuesta:

# 4 - Conclusiones (4 puntos)

* Compare los resultados obtenidos por cada modelo implementado.
* Indique aspectos positivos y negativos de cada enfoque de solución.

Respuesta:

El modelo RoBERTa-base fine-tuned obtiene los mejores resultados, con un F1-macro alto y un desempeño consistente. Su principal fortaleza es que está entrenado específicamente para la tarea, lo que le permite capturar mejor las diferencias entre hate speech y offensive language. Como desventaja, requiere datos etiquetados y un proceso de entrenamiento más costoso.

El modelo Llama-3.2-3B-Instruct con zero-shot prompting presenta un desempeño muy bajo. Tiende a colapsar en una sola clase y no logra distinguir correctamente entre las categorías, lo que evidencia que los modelos generativos, sin ajuste adicional, no son adecuados para tareas de clasificación supervisada. Su ventaja es que no requiere entrenamiento, pero a costa de una pérdida significativa de precisión.